Primero cargamos las bibliotecas y funciones de Python que vamos a usar.



In [1]:
import networkx as nx
from networkx.algorithms import isomorphism
from sympy.combinatorics.permutations import Permutation
from sympy.combinatorics.perm_groups import PermutationGroup
from math import factorial

from pycliques.lists import enlist_graphs
from pycliques.utilities import invert_dict
from pycliques.pictures import picture_list

def change_edge(g, e, ne):
    g1 = g.copy()
    g1.remove_edge(*e)
    g1.add_edge(*ne)
    return g1

def is_feasible_edge_replacement(g, e, ne):
    if nx.is_isomorphic(g, change_edge(g, e, ne)):
        GM = isomorphism.GraphMatcher(g, change_edge(g, e, ne))
        return list(GM.subgraph_isomorphisms_iter())
    else:
        return False

def dict_to_perm(d):
    """Change a dict d to a list the_list so that d[i] becomes the same as the_list[i]"""
    the_list = []
    for i in range(len(d)):
        the_list.append(d[i])
    return the_list

def perms_all_feasible_edge_replacements(graph):
    n = graph.order()
    perms = []
    edges = graph.edges()
    nonedges = [(x, y) for x in range(n) for y in range(n)
                if x != y and not (x, y) in edges]
    for e in edges:
        for ne in nonedges:
            isos = is_feasible_edge_replacement(graph, e, ne)
            if isos:
                perms.extend([Permutation(dict_to_perm(x)) for x in isos])
    return PermutationGroup(perms)

def local_amoebas_given_order(n):
    graphs = enlist_graphs(n)
    return [g for g in graphs if perms_all_feasible_edge_replacements(g).order() == factorial(n)]

Vamos a comprobar las funciones anteriores en una gráfica sencilla.



In [1]:
g = nx.path_graph(4)
nx.draw(g, with_labels=True)

En la gráfica anterior borramos la arista `01` e incluimos como
arista a `02`.



In [1]:
h = change_edge(g, (0, 1), (0, 2))
nx.draw(h, with_labels=True)

Verifiquemos que al hacer tales cambios no se obtiene una gráfica
isomorfa a la que habíamos empezado.



In [1]:
is_feasible_edge_replacement(g, (0, 1), (0, 2))

En caso de que las aristas se puedan cambiar, la función
`is_feasible_edge_replacement` regresa una lista de diccionarios de Python que
todos los isomorfismos posibles entre las gráficas etiquetadas. El
diccionario `{3: 0, 2: 3, 1: 2, 0: 1}` debe leerse, por ejemplo, que
el vértice 3 de la gráfica original debe enviarse al 0 de la nueva
gráfica, el 2 al 3, el 1 al 2 y el 0 al 1.



In [1]:
dicts = is_feasible_edge_replacement(g, (0, 1), (0, 3))
dicts

En el siguiente dibujo, se ponen juntas la nueva y la vieja gráfica
para compararlas.



In [1]:
h = change_edge(g, (0, 1), (0, 3))
nx.draw(nx.union(h, g, rename=('h-','g-')), with_labels=True)

Para reetiquetar la nueva gráfica de modo que quede etiquetada de
manera idéntica a la original, aplicamos el inverso de uno de los
mapeos, y comprobamos.



In [1]:
hrel = nx.relabel_nodes(h, invert_dict(dicts[1]))
nx.draw(hrel, with_labels=True)

Para trabajar con grupos en Python, usamos la biblioteca `sympy`. Cada
uno de los mapeos que obtuvimos puede convertirse en una permutación.



In [1]:
lperms = [dict_to_perm(x) for x in dicts]
lperms

La función `Permutation` recibe una lista como `[0,3,2,1]`, que
(en este caso) enlista las imágenes de 0,1,2,3. Es decir 0 se va a 0,
1 a 3, 2 a 2 y 3 a 1.



In [1]:
perms = [Permutation(x) for x in lperms]
perms

La función `perms_all_feasible_edge_replacements` regresa el grupo generado por
todas las permutaciones obtenidas de esta manera.



In [1]:
perms_all_feasible_edge_replacements(g)

La función `local_amoebas_given_order` toma un número entre el 6 y el 10 y
regresa la lista de las amoebas locales con ese orden.



In [1]:
amoebas6 = local_amoebas_given_order(6)
len(enlist_graphs(6)), len(amoebas6)

La función `picture_list` produce un dibujo de todas las gráficas en
una lista. (Funciona mejor si todas las gráficas en la lista son
conexas.) El primer argumento es una condición que deben satisfacer
las gráficas enlistadas en el segundo argumento para ser
graficadas. El tercer argumento es el tamaño del dibujo.



In [1]:
def return_true(graph):
    return True

picture_list(return_true, amoebas6, (6,6))

In [1]:
amoebas7 = local_amoebas_given_order(7)
len(enlist_graphs(7)), len(amoebas7)

In [1]:
picture_list(return_true, amoebas7, (12, 12))

Una dificultad del anterior método es que no es claro el índice de
cada amoeba en el dibujo. Con un método como el siguiente podemos
dibujar varias junto con su índice.



In [1]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,2.5))

for i in range(9):
    initial = 0
    plt.subplot(190+i+1)
    plt.title(str(i+initial))
    nx.draw(amoebas6[i+initial], with_labels=True)

plt.show()